In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
import math
import copy
import pandas as pd
import numpy as np
import itertools
import pickle
import time
import json
import os

In [51]:
df_chus = None

item_info = {}

unique_items = []
seq_util_val = []
util_list = []
seq_list = []

seq_no = {}
seq_pos_no = {}
seq_pos_no_util = {}
seq_pos_no_rem_util = {}
seq_pos_no_rbu = {}
seq_pos_no_lru = {}
seq_pos_no_rem_item_count = {}
item_seen_bef = [0] * 1570

item_neighbour = {}

temp_bitmap = []

keys_to_delete = []

item_info_dbv = {}

pattern_av_hash = {}

chus_hash = {}

visited = []
cluster_info = {}
cluster_info_per_reps = {}

dvf={}

In [52]:
def pattern_avoidable(item_l):

  _key_ = sum(dvf[tuple(item_l)]["RemItems"])
  swu_itemset_p = sum(dvf[tuple(item_l)]["SWU"])
  sup_itemset_p = len(dvf[tuple(item_l)]["SID"])


  if(_key_ not in pattern_av_hash):
    pattern_av_hash[_key_] = []

  mk = pattern_av_hash[_key_]
  #print(f"item_l: {item_l}")
  #print(f"mk: {mk})")
  #print()

  if(len(mk)==0):
    pattern_av_hash[_key_].append(item_l)
    #print(f"{item_l} is NEW")
    return False

  else:
    flag = 0
    for itemset_m in range(len(mk)):
      #print(f"itemset_m: {itemset_m} --- len-mk: {len(mk)}")
      #print(f"mk-itemset_m: {mk[itemset_m]}")
      #print()
      if(itemset_m >= len(mk)):
        break
      itemset_p_bar = mk[itemset_m]
      swu_itemset_p_bar = sum(dvf[tuple(itemset_p_bar)]["SWU"])              #item_info_dbv[tuple(itemset_p_bar)]['swu']
      sup_itemset_p_bar = len(dvf[tuple(itemset_p_bar)]["SID"])              #item_info_dbv[tuple(itemset_p_bar)]['support']

      if(swu_itemset_p_bar == swu_itemset_p and
         sup_itemset_p_bar == sup_itemset_p):
        l_p = len(item_l)
        l_p_bar = len(itemset_p_bar)
        if(l_p <= l_p_bar and item_l[l_p-1] == itemset_p_bar[l_p_bar-1]):
          #print(f"{item_l} is BACKWARD-SUB-PATTERN of {itemset_p_bar}")
          return True
        else:
          if(l_p > l_p_bar and item_l[l_p-1] == itemset_p_bar[l_p_bar-1]):
            #print(f"{item_l} is BACKWARD-SUPER-PATTERN of {itemset_p_bar}")
            del pattern_av_hash[_key_][itemset_m]
            flag = 1

    if(flag==1):
      pattern_av_hash[_key_].append(item_l)
      return True

  pattern_av_hash[_key_].append(item_l)
  #print(f"{item_l} is NEW -> last section")
  return False

In [53]:
def exp_siblings(item_l_child, s_i_temp, i_i_temp, minsup, minutil):
  #print("sib exp")
  chus_cand_set_sib = []
  for item_l_child_i in range(len(item_l_child)):
    item_l_child_i_main = item_l_child[item_l_child_i]
    chus_cand_set_sib += dfs_pattern_ext(item_l_child_i_main, s_i_temp, i_i_temp[item_l_child_i:], minsup, minutil)
  return chus_cand_set_sib

In [54]:
def dfs_pattern_ext(item_l,s_items,i_items,ms,mu):

  if(not pattern_avoidable(item_l)):
  #if True:

    s_i_temp = []
    i_i_temp = []
    item_l_child = []
    chus_cand_set = []

    for sindx in range(len(s_items)):
      s_i = s_items[sindx]
      key_of_last_item_neighbor = tuple([item_l[-1]])
      neighbor = dvf[key_of_last_item_neighbor]["Neighbors"]

      if((s_i not in neighbor) or (s_i in item_l)):
        continue
      else:
        key_si = tuple([s_i])
        key_iteml = tuple(item_l)

        iteml_sid = dvf[key_iteml]["SID"]
        iteml_event = dvf[key_iteml]["Event"]
        si_sid = dvf[key_si]["SID"]
        si_event = dvf[key_si]["Event"]

        iteml_si_pair = {}
        iteml_rlbu = 0

        for i in range(len(iteml_sid)):
          if(iteml_sid[i] in si_sid):
            si_sid_indx = si_sid.index(iteml_sid[i])
            if(si_event[si_sid_indx] > iteml_event[i]):
              iteml_si_pair_indx = [i, si_sid_indx]
              iteml_si_pair[iteml_sid[i]] = iteml_si_pair_indx
              iteml_rlbu += dvf[key_iteml]["RBU"][i]

        if(len(iteml_si_pair) >= ms and iteml_rlbu >= mu):
          s_i_temp.append(s_i)

        sid=[]
        eid=[]
        util = []
        rem_util = []
        swu = []
        rbu = []
        rem_item_count = []
        nbr = set()
        for key, value in iteml_si_pair.items():
          sid.append(key)
          eid.append(dvf[key_si]["Event"][value[1]])
          util.append(dvf[key_iteml]["Utility"][value[0]] + dvf[key_si]["Utility"][value[1]])
          rem_util.append(dvf[key_si]["RemUtil"][value[1]])
          swu.append(dvf[key_iteml]["SWU"][value[0]])
          tmp_rbu = dvf[key_iteml]["Utility"][value[0]] + dvf[key_si]["Utility"][value[1]] + dvf[key_si]["RemUtil"][value[1]]
          rbu.append(tmp_rbu)
          rem_item_count.append(dvf[key_si]["RemItems"][value[1]])

        if(sum(rbu)>=mu):
            iteml_ch = copy.deepcopy(item_l)
            iteml_ch.append(s_i)
            item_l_child.append(iteml_ch)
            dvf[tuple(iteml_ch)] = {
              "SID": sid,
              "Event": eid,
              "Utility": util,
              "RemUtil": rem_util,
              "SWU": swu,
              "RBU": rbu,
              "RemItems": rem_item_count,
              "Neighbors": nbr,
              "Flag": 1
            }

    chus_cand_set += item_l_child
    chus_cand_set += exp_siblings(item_l_child, s_i_temp, s_i_temp, ms, mu)

    return chus_cand_set

  else:
    return []

In [55]:
def is_sublist(sub_list, super_list):
  """Checks if a sub-list is contained within a super-list."""
  #if sub_list == super_list: return False
  return all(x in super_list for x in sub_list)

In [56]:
def elim_non_closed(cand_set, minutl):
  chus_set = []
  key_found_bef = {}
  for ik in range(len(cand_set)):
    if isinstance(cand_set[ik], int):
       its = cand_set[ik]
       itl = []
       itl.append(its)
       item_l = itl
    else:
      item_l = cand_set[ik]
    if(sum(dvf[tuple(item_l)]["Utility"])>=minutl):
      key = sum(dvf[tuple(item_l)]["SID"])
      if(key not in key_found_bef):
        chus_hash[key] = []
        chus_hash[key].append(item_l)
        key_found_bef[key] = 1
      else:
        chus_hash[key].append(item_l)
        key_found_bef[key] += 1

  for key, value in chus_hash.items():
    val1 = copy.deepcopy(value)
    val2 = copy.deepcopy(value)
    for i1 in val1:
      f=0
      for i2 in reversed(val2):
        if((i1 != i2) and (len(dvf[tuple(i1)]["SID"]) == len(dvf[tuple(i2)]["SID"]))):
          if(is_sublist(i1,i2)):
            f=1
            break
          elif(is_sublist(i2,i1)):
            indx = val2.index(i2)
            del val2[indx]
            del val1[indx]

      if(f==0):
        chus_set.append(i1)

  return chus_set

In [57]:
def CloHusp(ms,mu):

  filename = '/content/drive/MyDrive/MS Thesis/CHUS-6/Dataset/Kosarak10k-M.txt'

  file = open(filename, 'r')

  unique_item_set = set()

  j=0
  total_util = 0
  rem_items = []

  for line in file:
    #if(j==20):break
    row = [int(x) for x in line.replace(":", " ").split()]

    m = math.floor(len(row)/2)
    seq_util_val.append(row[m])
    tran_util = row[m]
    total_util += row[m]
    ul = row[m+1:]
    sl = row[:m]
    temp_set = set(sl)
    unique_item_set.update(temp_set)
    j+=1

    for i in range (0, len(sl)):
      item = sl[i]
      key = tuple([item])
      if key not in dvf:
        dvf[key] = {
          "SID": [],
          "Event": [],
          "Utility": [],
          "RemUtil": [],
          "SWU": [],
          "RBU": [],
          "RemItems": [],
          "Neighbors": set(),
          "Flag": 0
        }

        dvf[key]["SID"].append(j)
        dvf[key]["Event"].append(i+1)
        dvf[key]["Utility"].append(ul[i])
        dvf[key]["RemUtil"].append(sum(ul[i+1:]))
        dvf[key]["SWU"].append(tran_util)
        dvf[key]["RBU"].append(sum(ul[i:]))
        dvf[key]["RemItems"].append(len(sl)-(i+1))
        dvf[key]["Neighbors"].update(set(sl[i+1:]))

        if(dvf[key]["Flag"]==0 and len(dvf[key]["SID"])>=ms and sum(dvf[key]["SWU"])>=mu):
          rem_items.append(key[0])
          dvf[key]["Flag"] = 1

      else:
        dvf[key]["SID"].append(j)
        dvf[key]["Event"].append(i+1)
        dvf[key]["Utility"].append(ul[i])
        dvf[key]["RemUtil"].append(sum(ul[i+1:]))
        dvf[key]["SWU"].append(tran_util)
        dvf[key]["RBU"].append(sum(ul[i:]))
        dvf[key]["RemItems"].append(len(sl)-(i+1))
        dvf[key]["Neighbors"].update(set(sl[i+1:]))

        if(dvf[key]["Flag"]==0 and len(dvf[key]["SID"])>=ms and sum(dvf[key]["SWU"])>=mu):
          rem_items.append(key[0])
          dvf[key]["Flag"] = 1

  file.close()
  #mu = int(total_util*mu_perc)

  chus_cand = []
  chus_cand += rem_items

  for item_i in range(len(rem_items)):
    chus_cand += dfs_pattern_ext([rem_items[item_i]],rem_items,rem_items[(item_i+1):],ms,mu)

  print()
  print(f"Total no of Cand. CHUSPs: {len(chus_cand)}")
  print()

  chus_set = elim_non_closed(chus_cand,mu)

  print()
  print(f"Total no of CHUSPs: {len(chus_set)}")
  print()



  return chus_cand, chus_set

In [ ]:
"""
Tx(10k)     MS            MU Perc          MU         Cand.       CHUS       Time        Rep-CHUS         Eps        MinPts
------------------------------------------------------------------------------------------------------------------------------
~10k      400 (40%~)      ~0.03 %         42938        40          10         62 s           6            1.10          1
~10k      300 (30%~)      ~0.02 %         28625        63          19        113 s           8            1.10          2
~10k      200 (20%~)      ~0.01 %         14312       123          59        374 s          11            1.30          3
~10k      150 (10%~)      ~0.9  %         18881       381          73        642 s          13            1.20          4
"""

In [59]:
%%time
chus_ca, chus = CloHusp(150,18881)


Total no of Cand. CHUSPs: 539


Total no of CHUSPs: 29

CPU times: user 10.9 s, sys: 37.3 ms, total: 10.9 s
Wall time: 11.4 s


# **Candidate CHUSPs**

---



In [60]:
sc=[]
uc=[]
for i in range(len(chus_ca)):
  if(isinstance(chus_ca[i], int)):
    ck = [chus_ca[i]]
  else:
    ck = chus_ca[i]
  sc += [len(dvf[tuple(ck)]["SID"])]
  uc += [sum(dvf[tuple(ck)]["Utility"])]
df_chusc = pd.DataFrame({'CHUS Candidate - Patterns': chus_ca, 'Support': sc, 'Utility': uc})
df_chusc

,CHUS Candidate - Patterns,Support,Utility
0,6,6058,165910
1,3,4569,25470
2,11,3647,20129
3,1,1959,21520
4,7,843,23225
...,...,...,...
534,"[56, 303]",18,388
535,"[56, 90]",44,1220
536,"[56, 25]",28,1335
537,"[314, 6]",141,4570


# **CHUSPs**

---



In [61]:
sc=[]
uc=[]
for i in range(len(chus)):
  if(isinstance(chus[i], int)):
    ck = [chus[i]]
  else:
    ck = chus[i]
  sc += [len(dvf[tuple(ck)]["SID"])]
  uc += [sum(dvf[tuple(ck)]["Utility"])]
df_chus = pd.DataFrame({'CHUS Patterns': chus, 'Support': sc, 'Utility': uc})
df_chus

,CHUS Patterns,Support,Utility
0,[6],6058,165910
1,[3],4569,25470
2,[11],3647,20129
3,[1],1959,21520
4,[7],843,23225
5,[55],631,34720
6,"[6, 3]",2676,88163
7,"[6, 7]",730,40180
8,"[6, 148]",647,25486
9,"[6, 55]",391,32135


In [ ]:
dvf